In [33]:
%%writefile mapper.py


#!/usr/bin/python3

import sys
import re


# path = '/datasets/stop_words_en.txt'
path = 'stop_words_en.txt'

# Your code for reading stop words here

with open(path, 'r', encoding='utf-8') as f:
    stop_words = set(f.read().split())
    for line in f.readlines():
        stop_words.append(line.strip())


for line in sys.stdin:
    try:
        article_id, text = re.split('\t', line.strip(), 1)
    except ValueError as e:
        continue
    
    text = text.lower()
    article_id = int(article_id)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    if text.find(sys.argv[1]) >= 0:
        sys.stderr.write("reporter:counter:Wiki stats,Dt,1\n")
    sys.stderr.write("reporter:counter:Wiki stats,TOTAL,1\n")
        
    if article_id == 12:
        for word in words:
            word = word.strip()
            if word not in stop_words:
                sys.stderr.write("reporter:counter:Wiki stats,N_,1\n")
#                 if word.find(sys.argv[1]) >= 0:
                if word == sys.argv[1].strip():
                    sys.stderr.write("reporter:counter:Wiki stats,Nt,1\n")
#         if word.lower() in stop_words:
#             sys.stderr.write(f"reporter:counter:Wiki stats, {word.lower()},1\n")

Overwriting mapper.py


In [ ]:
%%writefile counter_process.py

#!/usr/bin/python3

import sys
import re
from math import log


# Your functions may be here.


sw = 0.0
tw = 0.0
if __name__ == '__main__':
    for line in sys.stdin:
        if line.strip().find("Dt") == 0:
            Dt = float(re.search('(?<==)[0-9]*'.format("Dt"), line)[0])
        if line.strip().find("N_") == 0:
            N = float(re.search('(?<==)[0-9]*'.format("N_"), line)[0])
        if line.strip().find("Nt") == 0:
            Nt = float(re.search('(?<==)[0-9]*'.format("Nt"), line)[0])
#     sys.stdout.write(f'Dt={Dt}, Nt={Nt},N={N}\n')
    sys.stdout.write('{:0.6f}'.format((Nt/N)*(1./log(1.+Dt))))

Overwriting counter_process.py


In [ ]:
%%bash

OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
# OUT_DIR="td_idf_task"
NUM_REDUCERS=8
LOGS="stderr_logs.txt"


# Stub code for your job

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopwords" \
    -D mapreduce.job.reduces=0 \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python3 mapper.py labor" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS

# hdfs dfs -cat ${OUT_DIR}/* | head

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

cat $LOGS | python3 ./counter_process.py

cat $LOGS >&2

In [ ]:
# !hdfs dfs -copyToLocal /data/wiki/en_articles_part

In [ ]:
# Dt = 0
# N=0
# Nt=0
# import re

# path = '/datasets/stop_words_en.txt'

# stop_words = []
# with open(path, 'r') as f:
#     for line in f.readlines():
#         stop_words.append(line.strip().lower())
        
# with open("en_articles_part/articles-part", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         if line.lower().find("labor")>=0:
#             Dt += 1
#         article_id, text = re.split('\t', line.strip(), 1)
#         if article_id == "12":
#             text = text.lower()
#             text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
#             text = re.sub("\W*\s+\W*", " ", text, flags=re.UNICODE)
# #             text = re.sub("\W+(?=\s+)|(?=\s+)\W+", "", text, flags=re.UNICODE)
# #             words = re.split("\W*\s+|\s+\W*|\W*\s+\W*", text, flags=re.UNICODE)
# #             words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
#             words = re.split("\s+", text, flags=re.UNICODE)
#             for word in words:
#                 if word in stop_words:
#                     continue
#                 if word=="labor":
#                     Nt += 1
#                 print(word)
#                 N += 1
                
# print(f"Dt = {Dt}, N= {N}, Nt = {Nt}")